In [ ]:
# This cell is only for setting example parameter defaults - gets replaced by sidecar.
day_obs_min = "2024-10-24"
#day_obs_max = "2024-10-31"
day_obs_max = "Today"
#instrument = "latiss"  
instrument = "lsstcomcam"

# ConsDB Visit Metadata from {{ params.day_obs_min }} to {{ params.day_obs_max }}

In [ ]:
import os
import copy
from math import floor
import numpy as np
import healpy as hp
import matplotlib.pylab as plt
from cycler import cycler
import colorcet as cc

import pandas as pd
from IPython.display import display, Markdown, HTML

import datetime
from astropy.time import Time, TimeDelta
import astropy.units as u
import astropy
astropy.utils.iers.conf.iers_degraded_accuracy = 'ignore'

import datetime
import pytz

import datetime
from zoneinfo import ZoneInfo

try:
    tz = ZoneInfo(timezone)
    tz_utc = ZoneInfo("UTC")
except ZoneInfoNotFoundError:
    print("Timezone should be a string recognizable to `ZoneInfo`.")
    print("Using Chile/Continental (+UTC) backup.")
    tz = ZoneInfo("Chile/Continental")
    tz_utc = ZoneInfo("UTC")

from rubin_scheduler.site_models import Almanac
from rubin_scheduler.utils import Site

import requests
import urllib
import sqlalchemy
from lsst_efd_client import EfdClient
from lsst.summit.utils import ConsDbClient



# for scheduler snapshots
from urllib.parse import urlparse
from lsst.resources import ResourcePath


# at USDF or at summit?
if os.getenv("EXTERNAL_INSTANCE_URL", "") == "https://summit-lsp.lsst.codes":
    efd = 'summit_efd'
else:
    efd = 'usdf_efd'  
    os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"
# Not sure of summit consdb access, just use USDF for now
os.environ["LSST_CONSDB_PQ_URL"] = "http://consdb-pq.consdb:8080/consdb"
os.environ["no_proxy"] += ",.consdb"

%matplotlib inline

In [ ]:
# test that day_obs_min is a proper day_obs
try:
    test_day_obs = Time(f"{day_obs_min}T12:00:00", format='isot', scale='utc')
except ValueError:
    msg = "day_obs_min should be a date formatted as YYYY-MM-DD"
    raise ValueError(msg)

if day_obs_max == "Today":
    # Shift the 12hour offset following the definition of day_obs in https://sitcomtn-032.lsst.io/    
    # Drop the hours, minutes, seconds to get the ISO formatted day_obs
    day_obs_max = (Time.now() - TimeDelta(0.5, format='jd')).iso[:10]

elif day_obs_max == "Yesterday":
    # Shift the 12hour offset following the definition of day_obs in https://sitcomtn-032.lsst.io/
    # Drop the hours, minutes, seconds to get the ISO fromatted day_obs
    day_obs_max = (Time.now() - TimeDelta(1.5, format='jd')).iso[:10]

else:
    # test that day_obs is a proper day_obs    
    try:
        test_day_obs = Time(f"{day_obs_max}T12:00:00", format='isot', scale='utc')
    except ValueError:
        msg = "day_obs_max should be a date formatted as YYYY-MM-DD"
        raise ValueError(msg)

In [ ]:
minutes_to_days = 1./60/24
seconds_to_days = 1./60/60/24

day_min = Time(f"{day_obs_min}T12:00:00", format='isot', scale='utc')
day_max = Time(f"{day_obs_max}T12:00:00", format='isot', scale='utc')
one_day = TimeDelta(1, format='jd')
days = day_min + one_day * np.arange(0, (day_max - day_min).jd + 1)
day_obss = [d.iso[0:10] for d in days]

In [ ]:
## Almanac ## 
display(Markdown(f"## Almanac information for {day_obs_min} to {day_obs_max}"))
site = Site('LSST')
almanac = Almanac()
alm = {}
for day_obs in day_obss:
    alm[day_obs] = {}
    night_events = almanac.get_sunset_info(evening_date=day_obs, longitude=site.longitude_rad)
    civil_sunset = Time(night_events['sunset'], format='mjd', scale='utc') 
    sunset = Time(night_events['sun_n12_setting'], format='mjd', scale='utc') 
    sunrise = Time(night_events['sun_n12_rising'], format='mjd', scale='utc')
    night_length = (sunrise.mjd - sunset.mjd) * 24
    alm[day_obs]['civil sunset'] = civil_sunset.iso
    alm[day_obs]['sunset'] = sunset.iso
    alm[day_obs]['sunrise'] = sunrise.iso
    alm[day_obs]['moon rise'] = Time(night_events['moonrise'], format='mjd', scale='utc').iso
    alm[day_obs]['moon set'] = Time(night_events['moonset'], format='mjd', scale='utc').iso
    alm[day_obs]['night length'] = night_length.round(3)
    moon_phase = almanac.get_sun_moon_positions(sunset.mjd)['moon_phase']
    alm[day_obs]['moon phase'] = moon_phase.round(2)
alm = pd.DataFrame(alm)
display(alm.T)

In [ ]:
# Get visits from consdb

day_obs_int_min = int(day_obs_min.replace('-', ''))
day_obs_int_max = int(day_obs_max.replace('-', ''))

# Use the ConsDB Client, and add a couple of tries 
consdb = ConsDbClient()

## yes, this should be visit1 table, but that doesn't work for this schema query
## and at least for now, they're close enough to the same
keys1 = list(consdb.schema('lsstcomcam', 'cdb_lsstcomcam.exposure').keys())  
## also, when I tried this at first, the schema query below only returned unique keys anyway
kk = list(consdb.schema('lsstcomcam', 'cdb_lsstcomcam.visit1_quicklook').keys())
keys2 = [k for k in kk if k not in keys1]

qcols = ""
for col in keys2:
    qcols += f"q.{col},"
qcols = qcols.rstrip(",")

# Joining in the database should be preferable
join_query = f'''
    SELECT v.*, {qcols}
    FROM cdb_{instrument}.visit1 v
    INNER JOIN cdb_{instrument}.visit1_quicklook q
    ON q.visit_id = v.visit_id 
     WHERE v.day_obs >= {day_obs_int_min}
     and v.day_obs  <= {day_obs_int_max}
'''

# Querying separately and joining in pandas works 
visit_query = f'''
    SELECT * 
    FROM cdb_{instrument}.visit1
     WHERE day_obs >= {day_obs_int_min}
     and day_obs  <= {day_obs_int_max}
'''

quicklook_query = f'''
    SELECT q.*  FROM cdb_{instrument}.visit1_quicklook as q,
    cdb_{instrument}.visit1 as v
     WHERE q.visit_id = v.visit_id and 
     v.day_obs >= {day_obs_int_min} 
     and v.day_obs <= {day_obs_int_max}
'''


try:
    visits = consdb.query(visit_query).to_pandas()
except requests.HTTPError or requests.JSONDecodeError:
    # Try twice
    visits = consdb.query(visit_query).to_pandas()

quicklook = consdb.query(quicklook_query).to_pandas()

if len(visits) > 0:
    display(Markdown(f"Retrieved {len(visits)} visits from consdb"))

if len(quicklook) > 0:
    visits = visits.join(quicklook, on='visit_id', lsuffix='', rsuffix='_q')
    display(Markdown(f"And added quicklook stats"))

if len(visits) == 0:
    display(Markdown(f"No visits for {telescope} between {day_obs_min} to {day_obs_max} retrieved from consdb"))

In [ ]:
# Patch science_program if was None
values = (dict([[e,""] for e in ['science_program','target_name', 'observation_reason']]))
visits.fillna(value=values, inplace=True)

In [ ]:
c = None
if len(visits) > 0:
    groupcols = ['science_program', 'img_type', 'target_name', 'observation_reason', 'day_obs', 'visit_id'] 
    c = visits[groupcols].groupby(['science_program', 'img_type']).agg({'science_program' : ['first'],
                                                                        'target_name' : ['unique'], 
                                                                        'observation_reason' : ['unique'],
                                                                        'day_obs' : ['nunique'],
                                                                        'visit_id' : ['first', 'last', 'count']})
    display(Markdown(f"ConsDB Visits"))
    display(c)#.sort_values(by=['science_program', ('visit_id', 'first')]))

In [ ]:
# This might work .. to help translate test block numbers above into more meaningful programs
jira_base_url = "https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/v2/testCase/"
with open("/home/l/lynnej/.zephyr_token", "r") as f:
    zephyr_token = f.read().rstrip("\n")
zephyr_url = "https://api.zephyrscale.smartbear.com/v2/testcases/"
headers = {"Accept": "application/json",
           "Authorization": f"Bearer {zephyr_token}",
           "Content-Type": "application/json"} 

if len(visits) > 0:
    test_names = {}
    jira_urls = {}
    for science_program in visits.science_program.unique():
        if science_program is not None and science_program.startswith("BLOCK-T"):
            response = requests.get(url=zephyr_url + science_program, headers=headers)
            test_name = response.json()['name']
            jira_url = jira_base_url + science_program
            
            test_names[science_program] = test_name
            jira_urls[science_program] = jira_url

            display(Markdown(f"[{science_program}]({jira_url}) - {test_name} ({len(visits.query('science_program == @science_program'))} visits)"))